In [2]:
import datetime
import glob
import json
import pandas as pd
import pickle
import numpy as np
import requests

# 소상공인

In [3]:
smallbiz_dates = [201512, 
                  201603, 201606, 201609, 201612, 
                  201703, 201706, 201709, 201712,
                  201803, 201806, 201809, 201812, 
                  201903, 201906, 201909, 201912, 
                  202003, 202006]
smallbiz_date_index = {date: i for i, date in enumerate(smallbiz_dates)}

In [4]:
lines = []
smallbiz_no = {}

for date in smallbiz_dates:
    temp_df = pd.read_pickle('외부데이터/공공데이터포털/소상공인시장진흥공단_상가(상권)정보/smallbiz_roi_df_%d.pkl' % date)
    
    for i, row in temp_df.iterrows():
        if row['상가업소번호'] not in smallbiz_no:
            lines.append(row)
            smallbiz_no[row['상가업소번호']] = [date]
        else:
            smallbiz_no[row['상가업소번호']].append(date)
            
# smallbiz_df = pd.DataFrame(lines)
# smallbiz_df = smallbiz_df.set_index('상가업소번호')
# smallbiz_df.to_pickle('외부데이터/공공데이터포털/소상공인시장진흥공단_상가(상권)정보/smallbiz_roi_df_201812-202006.pkl')

# with open('외부데이터/공공데이터포털/소상공인시장진흥공단_상가(상권)정보/smallbiz_number-to-dates_201812-202006.pkl', 'wb') as file:
#     pickle.dump(file=file, obj=smallbiz_no)

In [5]:
smallbiz_df = pd.read_pickle('외부데이터/공공데이터포털/smallbiz_clean_df_201812-202006.pkl')

In [30]:
len(smallbiz_df)

71478

In [6]:
smallbiz_df.iloc[0]

상호            피자마루 약수점
시도               서울특별시
시군구                 중구
행정동                약수동
대분류                 음식
중분류              패스트푸드
소분류               피자전문
start_date      201812
end_date        202006
층정보                  1
index         19977290
Name: 0, dtype: object

In [8]:
start_dates = []

for index, row in smallbiz_df.iterrows():
    start_date = min(smallbiz_no[row['index']])
    
    start_dates.append(start_date)

In [10]:
start_dates[::1000]

[201512,
 201603,
 201512,
 201512,
 201512,
 201512,
 201512,
 201603,
 201603,
 201512,
 201512,
 201512,
 201512,
 201512,
 201803,
 201603,
 201603,
 201512,
 201612,
 201612,
 201612,
 201612,
 201706,
 201612,
 201612,
 201709,
 201709,
 201803,
 201712,
 201512,
 201512,
 201512,
 201512,
 201512,
 201512,
 201512,
 201512,
 201512,
 201512,
 201512,
 201603,
 201512,
 201603,
 201512,
 201512,
 201512,
 201512,
 201512,
 201603,
 201603,
 201612,
 201612,
 201612,
 201706,
 201703,
 201709,
 201712,
 201803,
 201512,
 201903,
 201512,
 201603,
 201512,
 201903,
 201903,
 201903,
 201903,
 201903,
 201909,
 202003,
 202006,
 202006]

In [12]:
smallbiz_df['start_date'] = start_dates

smallbiz_df.to_pickle('외부데이터/공공데이터포털/smallbiz_clean_df_201812-202006_start-201512.pkl')

In [14]:
approval_df = pd.read_pickle('외부데이터/지방행정 인허가 데이터/preproc_approval_roi_df.pkl')
approval_df.head()

,상호,시도,시군구,행정동,대분류,중분류,소분류,start_date,end_date,duration,...,have_been_휴업,is_accurate_end_date,통신판매,자동판매기,인허가목록,면적,종업원수,자본금,소상공인_index,인허가_index
0,피자마루 약수점,서울특별시,중구,약수동,음식,패스트푸드,피자전문,20101122,20200831,3570,...,True,True,False,False,"[(일반음식점, 경양식)]",NaN,NaN,NaN,19977290,"(119144,)"
1,청암회관,서울특별시,노원구,중계본동,음식,한식,한식/백반/한정식,19941210,20200831,9396,...,True,True,False,False,"[(일반음식점, 한식)]",NaN,NaN,NaN,20013190,"(134435,)"
2,은하노래광장주점,서울특별시,중구,황학동,음식,유흥주점,룸살롱/단란주점,20020801,20200831,6605,...,True,True,False,False,"[(유흥주점영업, 룸살롱)]",NaN,NaN,NaN,20613530,"(101008,)"
3,서울과학기술대학교제2학생식당,서울특별시,노원구,공릉2동,음식,기타음식업,구내식당/자급식음식점,20130801,20200831,2587,...,True,True,False,False,"[(위탁급식영업, 위탁급식영업)]",NaN,NaN,NaN,20819907,"(54112,)"
4,화성진열장,서울특별시,중구,광희동,소매,철물/난방/건설자재소매,진열장판매,20071024,20200831,4695,...,True,True,True,False,"[(통신판매업, 기타)]",NaN,NaN,NaN,22318339,"(182432,)"


In [15]:
category_df = pd.read_excel('외부데이터/공공데이터포털/소상공인_업종별_지역별_업소수_category.xlsx')
category_df.head()

,대분류,중분류,소분류,숫자,정책자금 지원제외 업종 여부,소상공인 여부,우리 카테고리,Unnamed: 7
0,관광/여가/오락,PC/오락/당구/볼링등,기원,15,NaN,ㅇ,NaN,NaN
1,관광/여가/오락,PC/오락/당구/볼링등,당구장,54,NaN,ㅇ,1.0,NaN
2,관광/여가/오락,PC/오락/당구/볼링등,볼링장,13,NaN,ㅇ,1.0,NaN
3,관광/여가/오락,PC/오락/당구/볼링등,비디오감상실,9,NaN,ㅇ,NaN,NaN
4,관광/여가/오락,PC/오락/당구/볼링등,오락용사격장,1,NaN,ㅇ,4.0,NaN


In [25]:
clean_category = {}

for i, row in category_df.iterrows():
    if pd.isnull(row['정책자금 지원제외 업종 여부']):
        clean_category[(row['대분류'], row['중분류'], row['소분류'])] = row['우리 카테고리']

In [20]:
clean_category

{('관광/여가/오락', 'PC/오락/당구/볼링등', '기원'),
 ('관광/여가/오락', 'PC/오락/당구/볼링등', '당구장'),
 ('관광/여가/오락', 'PC/오락/당구/볼링등', '볼링장'),
 ('관광/여가/오락', 'PC/오락/당구/볼링등', '비디오감상실'),
 ('관광/여가/오락', 'PC/오락/당구/볼링등', '오락용사격장'),
 ('관광/여가/오락', 'PC/오락/당구/볼링등', '인터넷PC방'),
 ('관광/여가/오락', 'PC/오락/당구/볼링등', '전자오락실'),
 ('관광/여가/오락', 'PC/오락/당구/볼링등', '탁구장'),
 ('관광/여가/오락', 'PC/오락/당구/볼링등', '포켓볼장'),
 ('관광/여가/오락', '놀이/여가/취미', '기타오락관련산업'),
 ('관광/여가/오락', '놀이/여가/취미', '낚시터'),
 ('관광/여가/오락', '무도/유흥/가무', '노래방'),
 ('관광/여가/오락', '스포츠/운동', '롤러스케이트장'),
 ('관광/여가/오락', '스포츠/운동', '스케이트장'),
 ('관광/여가/오락', '스포츠/운동', '실내스케이트장'),
 ('관광/여가/오락', '스포츠/운동', '실내야구연습장'),
 ('관광/여가/오락', '연극/영화/극장', '연극/음악/예술관련기타'),
 ('관광/여가/오락', '연극/영화/극장', '음악감상'),
 ('관광/여가/오락', '요가/단전/마사지', '단전호흡'),
 ('관광/여가/오락', '요가/단전/마사지', '스포츠마사지'),
 ('관광/여가/오락', '요가/단전/마사지', '요가/단식'),
 ('관광/여가/오락', '요가/단전/마사지', '지압원'),
 ('관광/여가/오락', '요가/단전/마사지', '체형미교실'),
 ('부동산', '부동산관련서비스', '부동산신탁'),
 ('부동산', '부동산중개', '부동산중개'),
 ('부동산', '평가/개발/관리', '부동산감정평가'),
 ('부동산', '평가/개발/관리', '부동산자문'),
 ('생활서비스', '개인

In [23]:
for i, row in approval_df.iterrows():
    if (row['대분류'], row['중분류'], row['소분류']) not in clean_category:
        approval_df = approval_df.drop(i)

In [26]:
approval_df['arbit_cat'] = [clean_category[(row['대분류'], row['중분류'], row['소분류'])] for i, row in approval_df.iterrows()]

In [29]:
approval_df.to_pickle('외부데이터/지방행정 인허가 데이터/approval_clean_roi_df.pkl')

In [31]:
len(approval_df)

16586